# Building and pruning of large combiner matrices
With, kernuller, it is easy to build large matrices for combiners of many inputs (within the bounds of your RAM).
Here, we show how to identify the amount of observations provided by those combiners, and some tools to reduce the number of outputs to its minimum, which is still a challenge.
## Evaluation of the number of independent outputs
The first step is to be able to evaluate the number of independant outputs.

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
#import kernuller_class as kernuller
import kernuller
import astropy.coordinates
import astropy.units as u

from time import time

Here, we use set up a number of random binary parameters that must cover the input (observed scene parameter space). This number should be larger than the number of outputs of the combiners tested. (Using random numbers helps making sure we do not fall into symmetry or spacial frequency tropes, but the analysis should also work with a grid of source positions). 

In [ ]:
nrands = 400
randrhos = 0.5+ 40*np.random.rand(nrands)
randthetas = 360*np.random.rand(nrands)
randconts = 400*np.ones(nrands)
params = np.array([[randrhos[i], randthetas[i], randconts[i]] for i in range(randrhos.shape[0])])

We build a specific `CHARA2` array that has 7 apertures in a non-redundant configuration.

For each size of array (3 to 7 apertures), we build a kernuller object, then use the `get_rank()` methods with the array of test parameters, once for the nulls, and once for the kernels, to get the number of dimensions of both the set of nulls and the set of kernel nulls.

The `get_rank()` method computes the combiner's outputs (or kernel-outputs) for the whole set of inputs. These outputs constitute a family of outputs. The number of independent outputs is the number of independent vectors in this family.

In [ ]:
CHARA2 = np.vstack((kernuller.CHARA, np.array([30.,30.])))
nas = np.arange(3,8)
nullranks = []
kernranks = []
sizes = []
for i in nas:
    print("Working on a %d -input combiner"%(i))
    kernuller.expected_numbers(i)
    print("=====================================================")
    mykernuller = kernuller.kernuller(CHARA2[:i], 3.6e-6)
    mykernuller.build_procedural_model(verbose=False)
    sizes.append(mykernuller.Np.shape[0])
    anullrank = mykernuller.get_rank(params=params, mode="")
    nullranks.append(anullrank)
    akernelrank = mykernuller.get_rank(params=params, mode="kernels")
    kernranks.append(akernelrank)
    print("Found a null rank of %d"%(anullrank))
    print("Found a kernel rank of %d"%(akernelrank))
    print("=====================================================")
sizes = np.array(sizes)
nullranks = np.array(nullranks)
kernranks = np.array(kernranks)

### Now let's make a pretty plot of our results...

In [ ]:
thnas = np.arange(3,18)
fig = plt.figure(figsize=(7.5, 5.5))
plt.plot(nas[:-2], sizes[:-2],marker="X",
         linestyle="None",markersize=14, label="Number of nulls")

plt.plot(thnas, (thnas-1)*(thnas-2), "k--", label="Expected numbers")
plt.plot(nas, nullranks,marker="P",
         linestyle="None",markersize=10, label="Number of independent nulls")
plt.plot(thnas, 1/2*(thnas-1)*(thnas-2), "k--")
plt.plot(nas, kernranks,marker="o",
         linestyle="None",markersize=10, label="Number of independent kernel nulls")
#plt.plot(9, 8, color="C1", marker="s", label="9 apertures cascaded nulls")
#plt.plot(9, 4, color="C2", marker="s", label="9 apertures cascaded kernel nulls")
extranas = nas[-2:]
extrasizes = sizes[-2:]
plt.plot(extranas, 70*np.ones_like(extranas), marker="X",color="C0",
         markersize=12, linestyle="None")
for anextranas, anextrasize  in zip(extranas, extrasizes):
    print(anextranas,anextrasize)
    plt.arrow(anextranas, 70, 0, 5, head_width=0.2, head_length=5, color="C0")
    plt.text(anextranas, 70+2, str(anextrasize),color="C0")
plt.legend(loc="lower right")
plt.ylim(0, 80)
plt.xlim(2, 14)
plt.xticks(ticks=np.arange(2,15))
plt.xlabel(r"Number of inputs $n_a$")
plt.grid()
#plt.title("Growth of kernel nullers")
plt.show()

## Building reduced combiners
Now the possibility to reduce the combiners to those numbers is an entirely different matter. The delicate part is to make sure that the matrix we build ( by adding or removing rows ) remains the matrix of a lossless combiner.

The matrices of lossless combiners are semi-unitary to the left, meaning that that their conjugate-transpose is their left-inverse:
$$\mathbf{M}^H\mathbf{M} = \mathbf{I}.$$
This also means (equivalent) that all their singular values are ones.


The way we have used is implemented in the algorithm offerred in `generative_random_pruning()`.

The algorithm will try random ways to build a matrix, recording the steps it took (the recipe), and saving this recipe if the result is both lossless and complete (gives full rank *kernels*).



In [ ]:
statlocs = kernuller.CHARA
mykernuller = kernuller.kernuller(statlocs,3.6e-6)
mykernuller.build_procedural_model(verbose=False)

## One of the problems with this approach that inside the function, there is no good way to stop the search yet keep the recipes found...
For the 6T combination, you have to run it for around 2000 iterations if you want to be relatively sure to find something (10-15 minutes on a decent workstation).

In [ ]:
matrices, recipes = kernuller.generative_random_pruning(mykernuller.Ms, 2000)

In [ ]:
print("We found %d matrices, here is one:"%(len(matrices)))
matrices[0]

The best way to store those matrices is by using `np.save()`. However, numpy will pickle it, so when loading them, you have to make sure to allow pickle: `np.load("my/matrix.npy", allow_pickle=True)`

The legacy way to store the matrices is to store the recipe (which looks like that):

In [ ]:
recipes[0]

The matrix can then be recovered by sending the recipe to to a method that follows the same steps but in a deterministic manner following the recipe.

For this reason, it is super important that generative_random_pruning() and generative_from_recipe() remain in sync. When changing one, the other must be changed.

In [ ]:
kernuller.generative_from_recipe(mykernuller.Ms, recipes[0])